In [ ]:
# 미리 다운로드 받아야 할 파일
# pose_iter_440000.caffemodel, pose_deploy_linevec.prototxt

import os
import cv2
import sys
import numpy as np

# 모델 & 설정 파일
model = "../models/openPose/pose_iter_440000.caffemodel" #다운로드 링크 : http://posefs1.perception.cs.cmu.edu/OpenPose/models/pose/coco/pose_iter_440000.caffemodel
config = "../models/openPose/pose_deploy_linevec.prototxt" #다운로드 링크 : https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/models/pose/coco/pose_deploy_linevec.prototxt

files = [model, config]

#설정파일 있는지 체크
for f in files:
    if os.path.isfile(f) is False:
        print("모델 혹은 설정파일이 없습니다 : "+f)
        sys.exit()
        
# 포즈 점 개수, 점 연결 개수, 연결 점 번호 쌍
nparts = 18
npairs = 17
pose_pairs = [(1, 2), (2, 3), (3, 4),  # 왼팔
              (1, 5), (5, 6), (6, 7),  # 오른팔
              (1, 8), (8, 9), (9, 10),  # 왼쪽다리
              (1, 11), (11, 12), (12, 13),  # 오른쪽다리
              (1, 0), (0, 14), (14, 16), (0, 15), (15, 17)]  # 얼굴

# 테스트 이미지 파일
img_files = ["../images/pose1.jpg", "../images/pose2.jpg"]

# 네트워크 생성
net = cv2.dnn.readNet(model, config) #모델 불러오기
if net.empty():
    print("Net open failed!")
    sys.exit()

for f in img_files:
    img = cv2.imread(f)
    if img is None:
        continue

    # 블롭 생성 & 추론
    blob = cv2.dnn.blobFromImage(img, 1/255., (368, 368)) #이미지 크기, 컬러 평균값 지정
    net.setInput(blob)   #입력설정
    out = net.forward()  # out.shape=(1, 57, 46, 46)
    h, w = img.shape[:2]

    # 검출된 점 추출
    points = []  #좌표 정보를 저장할 list 생성

    for i in range(nparts):  # 0 ~ 17 까지 루프
        heatMap = out[0, i, :, :]  #  heatMap은 18개의 [46,46] 행렬
        _, conf, _, point = cv2.minMaxLoc(heatMap)  #heatMap의 최대값 위치, 최소값 사용안함
        x = int(w * point[0] / out.shape[3])   #최대값의 x,y 좌표 계산
        y = int(h * point[1] / out.shape[2])
        points.append((x, y) if conf > 0.1 else None)  # heat map threshold=0.1   #conf가 0.1 보다 크면 저장

    # 검출 결과 영상 만들기
    for pair in pose_pairs:
        p1 = points[pair[0]]
        p2 = points[pair[1]]
        if p1 is None or p2 is None:
            continue
        cv2.line(img, p1, p2, (0, 255, 0), 3, cv2.LINE_AA)
        cv2.circle(img, p1, 4, (0, 0, 255), -1, cv2.LINE_AA)
        cv2.circle(img, p2, 4, (0, 0, 255), -1, cv2.LINE_AA)
        
    cv2.imshow("img", img)
    cv2.waitKey()
cv2.destroyAllWindows()